Data Extraction

In [1]:
import pandas as pd

In [12]:
csv_file = 'E:\\BIG DATA BY ineuron\\PuntPartenerTestAssignment\\ASSIGNMENT\\Updated_sales_data.csv'

In [13]:
df = pd.read_csv(csv_file)

In [14]:
df.head(5)

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,176559,Bose SoundSport Headphones,1,99.99,04-07-2019 22:30,"682 Chestnut St, Boston, MA 02215"
2,176560,Google Phone,1,600,04-12-2019 14:38,"669 Spruce St, Los Angeles, CA 90001"
3,176560,Wired Headphones,1,11.99,04-12-2019 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"


Data Transformation

In [17]:
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'],errors='coerce')

In [18]:
df['Price Each'] = pd.to_numeric(df['Price Each'],errors='coerce')

In [19]:
df['total_sales'] = df['Quantity Ordered'] * df['Price Each']

In [21]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%y %H:%M', errors='coerce')


In [23]:
df['Order Year'] = df['Order Date'].dt.year

In [24]:
df['Order Month'] = df['Order Date'].dt.month

In [26]:
product_sales = df.groupby(['Product','Order Year','Order Month'])['total_sales'].sum().reset_index()

Data Loading

In [46]:
import sqlite3

In [47]:
conn = sqlite3.connect('sales_data.db')

In [29]:
df.to_sql('orders', conn, index=False, if_exists='replace')
product_sales.to_sql('product_sales', conn, index=False, if_exists='replace')


38

SQL Queries

In [48]:
conn = sqlite3.connect('sales_data.db')

In [49]:
cursor = conn.cursor()


In [55]:
query1 = """
SELECT "Product", SUM("total_sales") as "total_sales"
FROM "product_sales"
WHERE "Order Year" = (SELECT MAX("Order Year") FROM "product_sales")
AND "Order Month" >= (SELECT MAX("Order Month") FROM "product_sales") - 2
GROUP BY "product";
"""

In [56]:
cursor.execute(query1)
result1 = cursor.fetchall()
print("Query 1 Result:")
print(result1)


Query 1 Result:
[('20in Monitor', 16388.51), ('27in 4K Gaming Monitor', 105687.29000000001), ('27in FHD Monitor', 46496.9), ('34in Ultrawide Monitor', 91577.59), ('AA Batteries (4-pack)', 4362.24), ('AAA Batteries (4-pack)', 3614.9100000000003), ('Apple Airpods Headphones', 95250.0), ('Bose SoundSport Headphones', 54394.56), ('Flatscreen TV', 59700.0), ('Google Phone', 129000.0), ('LG Dryer', 14400.0), ('LG Washing Machine', 17400.0), ('Lightning Charging Cable', 13529.75), ('Macbook Pro Laptop', 300900.0), ('ThinkPad Laptop', 150998.49), ('USB-C Charging Cable', 10994.0), ('Vareebadd Phone', 36800.0), ('Wired Headphones', 9783.84), ('iPhone', 177800.0)]


In [32]:
query2 = """
SELECT product, SUM(total_sales) as total_sales
FROM product_sales
GROUP BY product
ORDER BY total_sales DESC
LIMIT 5;
"""

In [57]:
cursor.execute(query2)
result2 = cursor.fetchall()
print("Query 2 Result:")
print(result2)

Query 2 Result:
[('Macbook Pro Laptop', 792200.0), ('iPhone', 467600.0), ('ThinkPad Laptop', 389996.1), ('Google Phone', 339600.0), ('27in 4K Gaming Monitor', 244523.73)]


In [33]:
query3 = """
SELECT product, AVG(total_sales) as avg_sales
FROM product_sales
GROUP BY product;
"""

In [58]:
cursor.execute(query3)
result3 = cursor.fetchall()
print("Query 3 Result:")
print(result3)

Query 3 Result:
[('20in Monitor', 21228.07), ('27in 4K Gaming Monitor', 122261.865), ('27in FHD Monitor', 56246.25), ('34in Ultrawide Monitor', 118556.88), ('AA Batteries (4-pack)', 5452.799999999999), ('AAA Batteries (4-pack)', 4523.870000000001), ('Apple Airpods Headphones', 115575.0), ('Bose SoundSport Headphones', 65193.479999999996), ('Flatscreen TV', 71700.0), ('Google Phone', 169800.0), ('LG Dryer', 22200.0), ('LG Washing Machine', 18300.0), ('Lightning Charging Cable', 17498.975), ('Macbook Pro Laptop', 396100.0), ('ThinkPad Laptop', 194998.05), ('USB-C Charging Cable', 13479.599999999999), ('Vareebadd Phone', 43600.0), ('Wired Headphones', 11978.01), ('iPhone', 233800.0)]


Data Validation

In [44]:
missing_values = df.isnull().sum()
if any(missing_values):
    pass
    # Handle missing values appropriately (e.g., log and/or skip rows)

anomalies = df[(df['Quantity Ordered'] < 0) | (df['Price Each'] < 0)]
if not anomalies.empty:
    pass
    # Handle anomalies appropriately (e.g., correct values, log issues)

try:
    # Code for ETL
    pass 
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: {e}")